# Advanced Certification Program in Computational Data Science

##  A program by IISc and TalentSprint

### Assignment 7: Demand Forecasting

## Learning Objectives

At the end of the experiment, you will be able to :

* perform EDA and analyze the feature importance of sales data
* prepare the data for forecasting task
* solve the problem of Demand Planning Optimization
* implement ARIMA model to forecast the demands of a sales

## Information

Sales forecasting is an approach retailers use to anticipate future sales by analyzing past sales, identifying trends, and projecting data into the future. The simplest version of a sales forecast will look at sales in Store A during last year, assume a continuation of some multi-year trend for Store A (e.g. some percentage of growth or decline in sales), and project forward to predict sales in Store A this year.

Sales forecasting has been the standard approach to retailing from the beginning of the industry itself.

Although modern retailers have more data than ever (as well as new tools and business intelligence dashboards), and can easily leverage more advanced and accurate forms of forecasting, sales-based forecasting is still the backbone of most retail organizations.

some benefits of having an accurate sales forecast

* Improved decision-making about the future
* Reduction of sales pipeline and forecast risks
* Alignment of sales quotas and revenue expectations
* Reduction of time spent planning territory coverage and setting quota assignments
* Benchmarks that can be used to assess trends in the future
* Ability to focus a sales team on high-revenue, high-profit sales pipeline  opportunities, resulting in improved win rates


This Demand optimization can reduce operational costs by:

**Inventory Optimization:** matching store inventory with actual needs to reduce storage space needed (Rental Costs)

**Replenishment Optimization:** optimizing replenishment quantity per order to minimize the number of replenishments between warehouse and stores (Warehousing & Transportation Costs)


## Dataset

Store Item Demand Forecasting dataset consists of 5 years of store-item sales data from the year 2013 to 2017. It contains sales for 50 different items at 10 different stores with 913000 observations and 4 columns Date Store, item, sales.

Attributes information:

**Date:** Date starting from 2013 to 2017

**Store:** Store indicating with a number

**Item:** Item  in a store

**Sales:** No.of sales 

In [ ]:
#@title Download dataset
!pip -qq install pmdarima
!wget https://raw.githubusercontent.com/insaid2018/Term-3/master/Projects/Store_Item_demand_forecast.csv

### Import required packages 

In [ ]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from scipy import stats
from scipy.stats import normaltest
import pmdarima as pm

### Load the data

In [ ]:
dataset  =  pd.read_csv('Store_Item_demand_forecast.csv')
dataset.head(), dataset.shape

### Data Pre-processing

* Convert the Date column and separate Date, Month and Year columns to add more features including day of year,  day of week and week of year.

* Create a new feature $Day^{year}$ using Day of year and year



In [ ]:
# DATES FEATURES
def date_features(df):
    # Date Features
    dataset = df
    df['date'] = pd.to_datetime(dataset['date'])
    df['year'] = dataset.date.dt.year
    df['month'] = dataset.date.dt.month
    df['day'] = dataset.date.dt.day
    df['dayofyear'] = dataset.date.dt.dayofyear
    df['dayofweek'] = dataset.date.dt.dayofweek
    df['weekofyear'] = dataset.date.dt.weekofyear
    
    # Additionnal Data Features
    df['day^year'] = np.log((np.log(dataset['dayofyear'] + 1)) ** (dataset['year'] - 2000))
    
    # Drop date
    df.drop('date', axis=1, inplace=True)
    return df

In [ ]:
# Dates Features for Train
train = date_features(dataset)
train.head()

### ARIMA Analysis

#### Decomposition

Time series data can be broken down into four core components: the average value, a trend (i.e. an increasing mean), seasonality (i.e. a repeating cyclical pattern), and a residual (random noise). Trends and seasonality are not always present in time dependent data. The residual is what’s left over after trends and seasonality are removed. Time series models assume that the data is stationary and only the residual component satisfies the conditions for stationarity.

Python’s statsmodels library has a method for time series decomposition called `seasonal_decompose()`.

* Apply decomposition and plot the trend, seasonality and residual

In [ ]:
# Decomposition of time series
result = seasonal_decompose(train['sales'], model='additive', freq=365)

fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(15, 12)

In [ ]:
# Let's see it for 1000 records
result = seasonal_decompose(train['sales'][:1000], model='additive', freq=365)

fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(15, 12)

#### Stationarity

If a time series is stationary and has a particular behavior over a given time interval, then it is safe to assume that it will have same behavior at some later point in time. Most statistical modeling methods assume or require the time series to be stationary.

**What is variance?**

Variance is a measurement of the spread between numbers in a data set. The variance measures how far each number in the set is from the mean. The square root of variance is the standard deviation.

**How to check Whether data is Stationary or not?**

There are two ways we can check the stationarity of a time series. The first is by looking at the data. By visualizing the data it should be easy to identify a changing mean or variation in the data. For a more accurate assessment, there is the Dickey-Fuller test. If the ‘Test Statistic’ is greater than the ‘Critical Value’ then the time series is stationary.

Implement a function which takes a timeseries as input and perform:
  * Dicky fuller test and check p-value
  * Rolling mean and rolling standard and plot it with a graph

In [ ]:
# Create function to test stationarity
def test_stationarity(timeseries, window = 12):

    # Determing rolling statistics
    rolmean = timeseries.rolling(window).mean()
    rolstd = timeseries.rolling(window).std()

    # Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
test_stationarity(train['sales'])

Differencing the data with shift 1 and see the plot

In [ ]:
# differencing the data
diff = train.sales[:10000].diff(1).fillna(0)
test_stationarity(diff)

#### Dickey-Fuller test

In [ ]:
# Perform Dickey-Fuller test:
print('Results of Dickey-Fuller Test:')
dftest = adfuller(diff, autolag='AIC', maxlag = 20 )
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
pvalue = dftest[1]
if pvalue < 0.01:
    print('p-value = %.4f. The series is likely stationary.' % pvalue)
else:
    print('p-value = %.4f. The series is likely non-stationary.' % pvalue)

print(dfoutput)

### Autoregression Intuition

Consider a time series that was generated by an autoregression (AR) process with a lag of k.

We know that the ACF describes the autocorrelation between an observation and another observation at a prior time step that includes direct and indirect dependence information.

This means we would expect the ACF for the AR(k) time series to be strong to a lag of k and the inertia of that relationship would carry on to subsequent lag values, trailing off at some point as the effect was weakened.

We know that the PACF only describes the direct relationship between an observation and its lag. This would suggest that there would be no correlation for lag values beyond k.

This is exactly the expectation of the ACF and PACF plots for an AR(k) process.

### Moving Average Intuition

Consider a time series that was generated by a moving average (MA) process with a lag of k.

Remember that the moving average process is an autoregression model of the time series of residual errors from prior predictions. Another way to think about the moving average model is that it corrects future forecasts based on errors made in recent forecasts.

We would expect the ACF for the MA(k) process to show a strong correlation with recent values up to the lag of k, then a sharp decline to low or no correlation. By definition, this is how the process was generated.

For the PACF, we would expect the plot to show a strong relationship to the lag and a trailing off of correlation from the lag onwards.

Again, this is exactly the expectation of the ACF and PACF plots for an MA(k) process.

**Summary:** 
From the autocorrelation plot we can tell whether or not we need to add MA terms. From the partial autocorrelation plot, we know we need to add AR terms.

References:
https://machinelearningmastery.com/gentle-introduction-autocorrelation-partial-autocorrelation/

### How to interpret ACF and PACF plots

For time series models such as Auto Regression (AR), Moving Averages (MA), or their combinations (ARMA), we need to specify one or more parameters (eg. p, q). These can be obtained by looking at ACF (to infer q parameter) and PACF (to infer p parameter) plots.

In a nutshell:

* If the ACF plot declines gradually and the PACF drops instantly, use Auto Regressive model 
* If the ACF plot drops instantly and the PACF declines gradually, use Moving Average model 
* If both ACF and PACF decline gradually, combine Auto Regressive and Moving Average models (ARMA).
* If both ACF and PACF drop instantly (no significant lags), it’s likely we won’t be able to model the time series.

In [ ]:
# Visualize ACF and PACF plots
diff = train.sales[:10000].diff(1).fillna(0)
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(diff, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(diff, lags=40, ax=ax2)

Create and fit the ARIMA model:

In [ ]:
arima_mod6 = sm.tsa.ARIMA(train.sales[:10000], (8,1,1)).fit(disp=False)
arima_mod6.summary()

The summary attribute that results from the output of ARIMA returns a significant amount of information, 

* The coef column shows the weight (i.e. importance) of each feature and how each one impacts the time series. 
* The P>|z| column informs us of the significance of each feature weight. Here, each weight has a p-value lower or close to 0.05, so it is reasonable to retain all of them in our model.

Reading ACF and PACF plots is however still challenging, and using grid search to find optimal parameter values is sometimes useful. From grid search, we can obtain an optimal parameter combination that has the lowest error (such as MAPE) or lowest general quality estimator (such as AIC).


### Identifying optimal parameters using `auto_arima`

pmdarima‘s `auto_arima` function is extremely useful when building an ARIMA model as it helps us identify the most optimal p,d,q parameters and return a fitted ARIMA model.

**Using pmdarima for Auto ARIMA model:**

In the previous method, checking for stationarity, making data stationary if necessary, and determining the values of p and q using the ACF/PACF plots can be time-consuming and less efficient. Using pmdarima’s auto_arima() function makes this task easier for us by eliminating steps for implementing an ARIMA model.


#### auto_arima

The auto-ARIMA process seeks to identify the most optimal parameters for an ARIMA model, settling on a single fitted ARIMA model.

Auto-ARIMA works by conducting differencing tests (i.e., Kwiatkowski–Phillips–Schmidt–Shin, Augmented Dickey-Fuller or Phillips–Perron) to determine the order of differencing, d, and then fitting models within ranges of defined start_p, max_p, start_q, max_q ranges. If the seasonal optional is enabled, auto-ARIMA also seeks to identify the optimal P and Q hyper- parameters after conducting the Canova-Hansen to determine the optimal order of seasonal differencing, D.

In order to find the best model, auto-ARIMA optimizes for a given information_criterion, one of (‘aic’, ‘aicc’, ‘bic’, ‘hqic’, ‘oob’) (Akaike Information Criterion, Corrected Akaike Information Criterion, Bayesian Information Criterion, Hannan-Quinn Information Criterion, or “out of bag”–for validation scoring–respectively) and returns the ARIMA which minimizes the value.

**Note:** pmdarima may take a long time to find the terms for the dataset, subset of datapoints are used to minimize the execution time, however, we can experiment with a subset of the dataset.


From the ACF and PACF plot, we can observe that data is exhibiting same pattern for each 7 lags. (seasonal term 'm' = 7)

In [ ]:
# Fit the model
model = pm.auto_arima(train.sales[:1000], seasonal=True, m=7) # m is seasonal term

# Make forecast
forecasts = model.predict()  # predict 1 step into the future

In [ ]:
# Model summary
model.summary()